In [2]:
#Imports iniciais para o funcinamento do spark
import pyspark
from pyspark.sql import SparkSession

In [3]:
#Criando a spark session
spark = (SparkSession
         .builder
         .appName('TrabPratico-mod3')
         .getOrCreate())

21/09/12 11:12:21 WARN Utils: Your hostname, eduney-pc resolves to a loopback address: 127.0.1.1; using 172.26.67.202 instead (on interface eth0)
21/09/12 11:12:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/12 11:12:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [28]:
df_titles = (
    spark.read
    .csv('/mnt/c/Users/eduar/OneDrive/BootCamp EngDados/Modulo 3/data/title_basics.tsv',
         header=True,
         sep='\t',
         inferSchema=True)
    )
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [29]:
df_ratings = (
    spark.read
    .csv('/mnt/c/Users/eduar/OneDrive/BootCamp EngDados/Modulo 3/data/title_ratings.tsv',
         header=True,
         sep='\t',
         inferSchema=True)
    )
df_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
+---------+-------------+--------+
only showing top 5 rows



In [41]:
df_titles.createOrReplaceTempView('tab_titles')
df_ratings.createOrReplaceTempView('tab_ratings')

In [31]:
#Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?
(
    spark.sql(
        """
        Select count(1)
        from tab_titles
        where titleType in ('tvMovie','movie')
        and startYear = '2015'
        """
    )
).show()

+--------+
|count(1)|
+--------+
|   19987|
+--------+



In [38]:
#Qual o gênero de títulos mais frequente? 
(
    spark.sql(
        """
        Select genre
        , count(1)
        FROM (
            Select explode(split(genres,',')) as genre
            from tab_titles
        )
        group by genre
        order by count(1) desc
        """
    )
).show(5)

+-----------+--------+
|      genre|count(1)|
+-----------+--------+
|      Drama| 2247995|
|     Comedy| 1653725|
|      Short| 1021850|
|  Talk-Show|  900198|
|Documentary|  764885|
+-----------+--------+
only showing top 5 rows



In [44]:
#Qual o gênero com a melhor nota média de títulos? 
(
    spark.sql(
        """
        Select genre
        , avg(averageRating) as media_nota
        , count(tconst)
        FROM (
            Select t.tconst
            ,    r.averageRating
            ,    explode(split(t.genres,',')) as genre
            from tab_titles t
            inner join tab_ratings r on t.tconst = r.tconst
        )
        group by genre
        order by avg(averageRating) desc
        """
    )
).show(5)

+-----------+------------------+-------------+
|      genre|        media_nota|count(tconst)|
+-----------+------------------+-------------+
|    History| 7.353780102645086|        30396|
|Documentary| 7.240198535554575|       142443|
|  Biography|  7.17553191489362|        23124|
|    Mystery| 7.170086406897942|        54741|
|      Crime|7.1598428684859385|       119645|
+-----------+------------------+-------------+
only showing top 5 rows



In [60]:
#Qual o vídeo game do gênero aventura mais bem avaliado em 2020?
(
    spark.sql(
        """
        Select primaryTitle
         ,    avg(averageRating) as media_nota
        from (
            Select t.primaryTitle
            ,    r.averageRating
            ,    explode(split(t.genres,',')) as genre
            from tab_titles t
            inner join tab_ratings r on t.tconst = r.tconst
            where titleType = 'videoGame'
            and startYear = '2020'
            and array_contains(split(t.genres,','),'Adventure')
        )
        where genre = 'Adventure'
        group by primaryTitle
        order by avg(averageRating) desc
        """
    )
).show(5)

+--------------------+----------+
|        primaryTitle|media_nota|
+--------------------+----------+
|     Half-Life: Alyx|       9.5|
|   Ghost of Tsushima|       9.3|
|               Omori|       9.2|
|Ori and the Will ...|       9.1|
|Final Fantasy VII...|       9.1|
+--------------------+----------+
only showing top 5 rows



In [61]:
#Qual o vídeo game do gênero aventura mais bem avaliado em 2020?  
(
    spark.sql(
        """
        Select t.primaryTitle
        ,    avg(r.averageRating) as media_nota
        from tab_titles t
        inner join tab_ratings r on t.tconst = r.tconst
        where titleType = 'videoGame'
          and startYear = '2020'
          and array_contains(split(t.genres,','),'Adventure')
        group by t.primaryTitle
        order by avg(averageRating) desc
        """
    )
).show(5)

+--------------------+----------+
|        primaryTitle|media_nota|
+--------------------+----------+
|     Half-Life: Alyx|       9.5|
|   Ghost of Tsushima|       9.3|
|               Omori|       9.2|
|Ori and the Will ...|       9.1|
|Final Fantasy VII...|       9.1|
+--------------------+----------+
only showing top 5 rows



In [95]:
#Qual o percentual de títulos do gênero comédia lançados em 2018 em relação ao total de títulos lançados nesse ano? 
(
    spark.sql(
        """
        Select genre
        , count(1) as qtd
        , max(qtd_total) as qtd_total
        , (count(1) / max(qtd_total) * 100) as percentual
        from (
            Select primaryTitle
            , explode(split(genres,',')) as genre
            , qtd_total
            from (Select t.primaryTitle
                         ,t.genres
                         , count(t.tconst) OVER (PARTITION BY startYear) as qtd_total
                  from tab_titles t
                  where startYear = '2018'
                  )
        )
        where genre = 'Comedy'
        group by genre
        """
    )
).show()

+------+-----+---------+------------------+
| genre|  qtd|qtd_total|        percentual|
+------+-----+---------+------------------+
|Comedy|78809|   402244|19.592336989488967|
+------+-----+---------+------------------+



In [64]:
df_titles.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [40]:
df_ratings.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [6]:
spark.stop()